# Train model
* get a pretrained EfficientNetB2 model
* Do training on Food101 dataset
  * First try 20% data
  * Then try full dataset
* Save model to file

In [1]:
# import torch, torchvision modules
import torch
import torchvision
from torch import nn
from torchvision import transforms

print(f"torch {torch.__version__} | torchvision {torchvision.__version__}")

torch 2.0.1+cu118 | torchvision 0.15.2+cu118


In [2]:
# import torchinfo for model summary
try:
    from torchinfo import summary
except:
    !pip install torchinfo
    from torchinfo import summary

In [ ]:
# import modules from sources
from sources import utils, datasetup, engine, models

2023-11-21 18:04:43.780636: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-21 18:04:44.097440: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-21 18:04:45.011177: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# device agnostic. For training, we can use cpu or gpu depend on available hardware
# when deploying on gradio, we only have free cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# setup pretrained EfficientNet2 for 101 food classes
model, model_transforms = models.create_effnet(
    effnet_version=2,
    num_class_names=101
)
model_transforms

In [ ]:
# check summary of the model
summary(
    model=model,
    input_size=[32, 3, 224, 224],
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20
)

In [ ]:
# compose additional transformation for train dataset
train_transforms = transforms.Compose([
    transforms.TrivialAugmentWide(),
    model_transforms
])
train_transforms

In [ ]:
# setup dataloaders
from pathlib import Path
from torchvision import datasets

data_dir = Path("/gpfs/alpine/chm135/proj-shared/buu/tmp")

train_data = datasets.Food101(
    root=data_dir,
    split="train",
    transform=train_transforms,
    download=True
)

test_data = datasets.Food101(
    root=data_dir,
    split="test",
    transform=model_transforms,
    download=True
)

In [ ]:
# train class names
class_names = train_data.classes
class_names[:5]

In [ ]:
# setup function to get 20% of training data
def split_dataset(dataset: torchvision.datasets,
                 split_size: float=0.2):

    length_1 = int(len(dataset) * split_size)
    length_2 = len(dataset) - length_1
    
    random_split_1, random_split_2 = torch.utils.data.random_split(
        dataset=dataset,
        lengths=[length_1, length_2]
    )

    return random_split_1, random_split_2

In [ ]:
# get 20% of dataset for train and test set
train_data_20, _ = split_dataset(
    dataset=train_data,
    split_size=0.2
)

test_data_20, _ = split_dataset(
    dataset=test_data,
    split_size=0.2
)

print(f"train length: {len(train_data_20)} | test length: {len(test_data_20)}")

In [ ]:
# setup train and test dataloaders
import os
import torch
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

train_dataloader = torch.utils.data.DataLoader(
    dataset=train_data_20,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS
)

test_dataloader = torch.utils.data.DataLoader(
    dataset=test_data_20,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS
)

print(f"{train_dataloader} | {test_dataloader}")
print(f"train and test length: {len(train_dataloader)} | {len(test_dataloader)}")

train_dataloader, test_dataloader, len(train_dataloader), len(test_dataloader)

In [ ]:
# setup loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)

In [ ]:
# train the model using train function from engine module
results = engine.train(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    epochs=1,
    device=device
)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(10,4))
ax[0].plot(results["train_loss"], label="train_loss")
ax[0].plot(results["test_loss"], label="test_loss")
ax[0].legend()

ax[1].plot(results["train_acc"], label="train_acc")
ax[1].plot(results["test_acc"], label="test_acc")
ax[1].legend()

In [ ]:
# save train model to `saved_models`
saved_model_path = utils.save_model(
    model=model,
    model_dir="saved_models",
    model_name_grid=[model.name]
)

In [ ]:
# check the size of trained model
saved_model_path = Path("saved_models/EfficientNet_B2.pth")
model_size = Path(saved_model_path).stat().st_size // (1024*1024)
model_size

# Read model from source file

In [ ]:
# get pretrained model
model, model_transforms = models.create_effnet(
    effnet_version=2,
    num_class_names=101,
    device=torch.device("cpu")
)

next(iter(model.parameters())).device

In [ ]:
# check summary
summary(
    model=model,
    input_size=[32, 3, 224, 224],
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20
)

In [ ]:
# load model from file
model.load_state_dict(torch.load(
    f=saved_model_path,
    map_location=torch.device("cpu")
))

next(iter(model.parameters())).device

In [ ]:
# predict function
import random
from typing import Tuple, Dict, List
from timeit import default_timer as timer
import PIL

def predict(img: PIL.Image) -> Tuple[Dict, float]:

    # start timer
    start_time = timer()
    
    # convert image to torch.tensor
    img_transformed = model_transforms(img).unsqueeze(dim=0)

    # predict food label probability
    model.eval()
    with torch.inference_mode():
        pred_prob = torch.softmax(model(img_transformed), dim=1)

    pred_prob_dict = {
        class_names[i]: float(pred_prob[0][i]) for i in range(len(class_names))
    }

    # wall time for prediction
    pred_time = timer() - start_time

    return pred_prob_dict, pred_time

random_path = random.choice(list(data_dir.glob("*/*/*/*")))
img = PIL.Image.open(random_path)
predict(img)

In [ ]:
# write class names to file
with open("class_names.txt", "w") as f:
    f.write("\n".join(class_names))

In [ ]:
# read class name file to list
with open("class_names.txt", "r") as f:
    read_class_name = [cls.strip() for cls in f.readlines()]


In [ ]:
example_dir = Path("examples")
example_dir.mkdir(parents=True, exist_ok=True)
random_paths = random.sample(list(data_dir.glob("*/*/*/*")), k=3)

# copy three example to example directory
import shutil
for rp in random_paths:
    shutil.copy2(dst=example_dir, src=rp)

In [ ]:
try:
    import gradio as gr
except:
    !pip install typing-extensions --upgrade
    !pip install -q gradio
    import gradio as gr

In [ ]:
import torch
from sources import models

In [ ]:
# create demo directory
Path("demo").mkdir(parents=True, exist_ok=True)

In [ ]:
%%writefile demo/app.py

from typing import Tuple, Dict
import os
import gradio as gr
import PIL
# get pretrained model
model, model_transforms = models.create_effnet(
    effnet_version=2,
    num_class_names=101,
    device=torch.device("cpu")
)

# load model from file
model.load_state_dict(torch.load(
    f="saved_models/EfficientNet_B2.pth",
    map_location=torch.device("cpu")
))

# read class names from file
with open("class_names.txt", "r") as f:
    class_names = [cls.strip() for cls in f.readlines()]

def predict(img: PIL.Image) -> Tuple[Dict, float]:

    # start timer
    start_time = timer()
    
    # convert image to torch.tensor
    img_transformed = model_transforms(img).unsqueeze(dim=0)

    # predict food label probability
    model.eval()
    with torch.inference_mode():
        pred_prob = torch.softmax(model(img_transformed), dim=1)

    pred_prob_dict = {
        class_names[i]: pred_prob[0][i] for i in range(len(class_names))
    }

    # wall time for prediction
    pred_time = timer() - start_time

    return pred_prob_dict, pred_time


# create gradio app
example_list = [["examples/" + example] for example in os.listdir("examples")]

# Create Gradio interface 
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Label(num_top_classes=5, label="Predictions"),
        gr.Number(label="Prediction time (s)"),
    ],
    examples=example_list,
    title="big food vision model",
    description="101 food classes",
    article="",
)

# Launch the app!
demo.launch(share=True)

In [ ]:
torch.__version__

In [ ]:
torchvision.__version__

In [ ]:
gr.__version__

In [ ]:
%%writefile demo/requirements.txt
torch==2.0.1
torchvision==0.15.2
gradio==4.4.0